In [ ]:
import torch
import torch.nn

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torchtext
torchtext.__version__

'0.14.1'

In [ ]:
import torchtext.transforms as T
from torch.hub import load_state_dict_from_url

padding_idx = 1
bos_idx = 0
eos_idx = 2
max_seq_len = 256
xlmr_vocab_path = r"https://download.pytorch.org/models/text/xlmr.vocab.pt"
xlmr_spm_model_path = r"https://download.pytorch.org/models/text/xlmr.sentencepiece.bpe.model"

text_transform = T.Sequential(
    T.SentencePieceTokenizer(xlmr_spm_model_path),
    T.VocabTransform(load_state_dict_from_url(xlmr_vocab_path)),
    T.Truncate(max_seq_len-2),
    T.AddToken(token = bos_idx, begin = True),
    T.AddToken(token = eos_idx, begin = False)
)

In [ ]:
import torchtext
text_transform_pretrained = torchtext.models.XLMR_BASE_ENCODER.transform()

In [ ]:
text_transform

Sequential(
  (0): SentencePieceTokenizer()
  (1): VocabTransform(
    (vocab): Vocab()
  )
  (2): Truncate()
  (3): AddToken()
  (4): AddToken()
)

In [ ]:
text_transform_pretrained

Sequential(
  (0): SentencePieceTokenizer()
  (1): VocabTransform(
    (vocab): Vocab()
  )
  (2): Truncate()
  (3): AddToken()
  (4): AddToken()
)

In [ ]:
from torchtext.datasets import SST2

In [ ]:
batch_size = 16

In [ ]:
!pip install torchdata==0.5.1
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
train_datapipe = SST2(split="train")
dev_datapipe = SST2(split="dev")

In [ ]:
tt_datapipe = SST2(split="train")

In [ ]:
tt_datapipe

ShardingFilterIterDataPipe

In [ ]:
train_datapipe

ShardingFilterIterDataPipe

In [ ]:
torch.__version__

'1.13.1+cu116'

In [ ]:
train_datapipe, dev_datapipe

(ShardingFilterIterDataPipe, ShardingFilterIterDataPipe)

In [ ]:
next(iter(train_datapipe))

('hide new secretions from the parental units', 0)

In [ ]:
next(iter(dev_datapipe))

("it 's a charming and often affecting journey .", 1)

In [ ]:
from torch.utils.data import DataLoader
# Transform the raw dataset using non-batched API (i.e apply transformation line by line)
def apply_transform(x):
  return text_transform(x[0]), x[1]

train_datapipe = train_datapipe.map(apply_transform)
train_datapipe = train_datapipe.batch(batch_size)
train_datapipe = train_datapipe.rows2columnar(["token_ids", "target"])
train_dataloader = DataLoader(train_datapipe, batch_size=None)

dev_datapipe = dev_datapipe.map(apply_transform)
dev_datapipe = dev_datapipe.batch(batch_size)
dev_datapipe = dev_datapipe.rows2columnar(["token_ids", "target"])
dev_dataloader = DataLoader(dev_datapipe, batch_size=None)

tt_datapipe = tt_datapipe.map(apply_transform)
tt_datapipe = tt_datapipe.batch(batch_size)
tt_datapipe = tt_datapipe.rows2columnar(["token_ids", "target"])
tt_dataloader = DataLoader(tt_datapipe, batch_size=None)

In [ ]:
for batch in dev_dataloader:
  print(batch)
  break

{'token_ids': [[0, 442, 242, 7, 10, 108654, 214, 136, 27983, 52490, 214, 120696, 6, 5, 2], [0, 51, 36361, 19, 59207, 538, 2586, 344, 136, 145342, 13, 2], [0, 114864, 1821, 47, 15673, 450, 110, 1414, 83, 5737, 297, 47, 352, 1299, 92, 10, 13036, 80997, 237, 10, 60091, 14373, 26868, 5844, 1346, 55474, 6, 5, 2], [0, 70, 1030, 1916, 6, 4, 84659, 7, 6, 4, 19612, 6, 4, 27076, 18, 87168, 136, 45730, 621, 756, 237, 16450, 33770, 34475, 70, 36049, 242, 7, 1312, 18864, 83221, 6, 5, 2], [0, 442, 242, 7, 72803, 4210, 4552, 6, 4, 4552, 72803, 6, 5, 2], [0, 102971, 21, 37534, 678, 16000, 136, 10, 10846, 1207, 318, 7844, 116281, 7, 6, 4, 70, 1346, 83, 10, 210651, 214, 538, 85583, 6713, 99, 27150, 24793, 6, 5, 2], [0, 10, 68018, 120, 5670, 223, 1346, 6, 5, 2], [0, 707, 20594, 4568, 6602, 242, 7, 152838, 678, 935, 1119, 9, 2452, 2242, 6, 5, 2], [0, 398, 54, 653, 25, 18, 765, 47, 3714, 1672, 19612, 47, 77947, 70, 1346, 242, 7, 23468, 519, 214, 171759, 111, 180327, 136, 99691, 6, 5, 2], [0, 23, 66161, 178

/usr/local/lib/python3.8/dist-packages/torch/utils/data/datapipes/iter/combining.py:262: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [ ]:
for batch in tt_dataloader:
  print(batch)
  break

{'token_ids': [[0, 1274, 112, 3525, 23410, 17514, 1295, 70, 49129, 289, 25072, 7, 2], [0, 70541, 7, 110, 43198, 6, 4, 4734, 27554, 71, 914, 9405, 2], [0, 450, 5161, 7, 6863, 124850, 136, 6, 127219, 1636, 9844, 43257, 34923, 1672, 14135, 31425, 2], [0, 47143, 7, 75, 3055, 538, 214521, 47, 47143, 70, 5701, 87420, 2], [0, 98, 70, 130481, 52758, 429, 9, 4390, 9, 2347, 9, 1679, 6468, 221115, 7, 70, 1346, 114014, 5809, 17745, 61007, 1257, 2], [0, 450, 242, 7, 2060, 5792, 187641, 47, 75688, 6044, 158788, 39734, 2], [0, 106804, 90, 450, 70, 14364, 111, 6044, 206686, 46389, 978, 37499, 237, 65842, 27528, 831, 7464, 15504, 1810, 10, 19336, 6, 4, 3357, 1346, 678, 142, 88965, 58982, 2146, 6, 5, 2], [0, 111, 858, 2408, 2], [0, 10, 8, 11856, 297, 809, 18266, 33, 9, 46799, 9, 18345, 242, 7, 215612, 141, 30260, 1294, 2], [0, 621, 1286, 53894, 538, 17569, 8305, 3501, 23, 2684, 9473, 7108, 9, 201950, 214, 242, 54180, 2], [0, 60899, 47, 48152, 140909, 7, 2], [0, 100, 8382, 14277, 519, 1314, 2750, 186992,

In [ ]:
num_classes = 2
input_dim = 768

from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER

classifier_head = RobertaClassificationHead(num_classes = 2, input_dim = input_dim)
model = XLMR_BASE_ENCODER.get_model(head = classifier_head)
model.to(device)

Downloading: "https://download.pytorch.org/models/text/xlmr.base.encoder.pt" to /root/.cache/torch/hub/checkpoints/xlmr.base.encoder.pt


  0%|          | 0.00/1.03G [00:00<?, ?B/s]

RobertaModel(
  (encoder): RobertaEncoder(
    (transformer): TransformerEncoder(
      (token_embedding): Embedding(250002, 768, padding_idx=1)
      (layers): TransformerEncoder(
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (linear1): Linear(in_features=768, out_features=3072, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=3072, out_features=768, bias=True)
            (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
          (1): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj)

In [ ]:
import torchtext.functional as F
from torch.optim import AdamW
from torch import nn

lr = 1e-5
optimizer = AdamW(model.parameters(), lr = lr)
loss_fn = nn.CrossEntropyLoss()

def train_step(input, target):
  output = model(input)
  loss = loss_fn(output, target)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

def eval_step(input, target):
  output = model(input)
  loss = loss_fn(output, target)
  return loss, (output.argmax(1) == target).type(torch.float).sum()

def evaluating():
  model.eval()
  total_loss = 0
  correct_predictions = 0
  total_predictions = 0
  counter = 0
  with torch.inference_mode():
    for batch in dev_dataloader:
      input = F.to_tensor(batch["token_ids"], padding_value = padding_idx).to(device)
      target = torch.tensor(batch["target"]).to(device)
      loss, predictions = eval_step(input, target)
      total_loss += loss
      correct_predictions += predictions
      total_predictions += len(target)
      counter += 1
      break
  return total_loss/counter, correct_predictions/total_predictions


In [ ]:
from tqdm import tqdm
epochs = 1
for epoch in tqdm(range(epochs)):
  for batch in train_dataloader:
    input = F.to_tensor(batch["token_ids"], padding_value=padding_idx).to(device)
    target = torch.tensor(batch["target"]).to(device)
    train_step(input, target)

  loss, accuracy = evaluating()
  print(f"Epoch : {epoch} | loss : {loss} | accuracy : {accuracy}")

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/modules/transformer.py:276: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:175.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)
100%|██████████| 1/1 [15:28<00:00, 928.78s/it]

Epoch : 0 | loss : 0.06217917799949646 | accuracy : 1.0


In [ ]:
string = "It's very bad"

In [ ]:
text = (string, -9999999)
text

("It's very bad", -9999999)

In [ ]:
from torchdata.datapipes.iter import IterableWrapper
dp = IterableWrapper([text])
dp

IterableWrapperIterDataPipe

In [ ]:

dp = dp.sharding_filter()

In [ ]:
dp

ShardingFilterIterDataPipe

In [ ]:
next(iter(dp))

("It's very bad", -9999999)

In [ ]:
dp = dp.map(apply_transform)
dp = dp.batch(1)
dp = dp.rows2columnar(["token_ids", "target"])
dp = DataLoader(dp, batch_size=None)

In [ ]:
for batch in dp:
  print(batch)

{'token_ids': [[0, 1650, 25, 7, 4552, 6494, 2]], 'target': [-9999999]}


In [ ]:
val = next(iter(dp))
val

{'token_ids': [[0, 1650, 25, 7, 4552, 6494, 2]], 'target': [-9999999]}

In [ ]:
batch['token_ids']

[[0, 1650, 25, 7, 4552, 6494, 2]]

In [ ]:
input = F.to_tensor(batch["token_ids"], padding_value = padding_idx).to(device)

In [ ]:
model.eval()
with torch.inference_mode():
  output = model(input)

In [ ]:
output

tensor([[ 2.5512, -2.1419]], device='cuda:0')

In [ ]:
ans = output.argmax(1)

In [ ]:
ans

tensor([0], device='cuda:0')

In [ ]:
ans.item()

1

In [ ]:
 try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning

In [ ]:
from google.colab import files
files.download('xlmr_base_encoder.zip')

In [ ]:
!cd models/ && zip -r ../xlmr_base_encoder.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

  adding: xlmr_base_encoder.pth (deflated 29%)


In [ ]:
# Save model
from going_modular.going_modular import utils

utils.save_model(
    model = model,
    target_dir = "models",
    model_name = "xlmr_base_encoder.pth")

[INFO] Saving model to: models/xlmr_base_encoder.pth


In [ ]:
# Setup best model filepath
model_path = "models/xlmr_base_encoder.pth"

#  load in the saved state_dict()
classifier_head = RobertaClassificationHead(num_classes = 2, input_dim = input_dim)
model = XLMR_BASE_ENCODER.get_model(head = classifier_head)

# Load the save model state_dict()
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
model.to('cuda')

RobertaModel(
  (encoder): RobertaEncoder(
    (transformer): TransformerEncoder(
      (token_embedding): Embedding(250002, 768, padding_idx=1)
      (layers): TransformerEncoder(
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (linear1): Linear(in_features=768, out_features=3072, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=3072, out_features=768, bias=True)
            (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
          (1): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj)

In [ ]:
next(model.parameters()).is_cuda

True

In [ ]:
class_names = ['bad', 'Good']

In [ ]:
from torchdata.datapipes.iter import IterableWrapper
from torch.utils.data import DataLoader
import torchtext.functional as F
from timeit import default_timer as timer
# from typing import Tuple, Dict

def predict(string):

  start_time = timer()

  var = (string, -9999999)
  dp = IterableWrapper([var])
  dp = dp.sharding_filter()

  padding_idx = 1
  bos_idx = 0
  eos_idx = 2
  max_seq_len = 256
  xlmr_vocab_path = r"https://download.pytorch.org/models/text/xlmr.vocab.pt"
  xlmr_spm_model_path = r"https://download.pytorch.org/models/text/xlmr.sentencepiece.bpe.model"

  text_transform = T.Sequential(
      T.SentencePieceTokenizer(xlmr_spm_model_path),
      T.VocabTransform(load_state_dict_from_url(xlmr_vocab_path)),
      T.Truncate(max_seq_len-2),
      T.AddToken(token = bos_idx, begin = True),
      T.AddToken(token = eos_idx, begin = False)
  )

  # Transform the raw dataset using non-batched API (i.e apply transformation line by line)
  def apply_transform(x):
    return text_transform(x[0]), x[1]

  dp = dp.map(apply_transform)
  dp = dp.batch(1)
  dp = dp.rows2columnar(["token_ids", "target"])
  dp = DataLoader(dp, batch_size=None)

  val = next(iter(dp))
  model.to('cpu')
  value = F.to_tensor(val["token_ids"], padding_value = padding_idx).to('cpu')
    # Pass transformed image through the model and turn the prediction logits into probabilities
  model.eval()
  with torch.inference_mode():
    answer = model(value)
  print(answer)
  # answer = answer.argmax(1)
  answer = torch.softmax(answer, dim=1)
  pred_labels_and_probs = {class_names[i]: float(answer[0][i]) for i in range(len(class_names))}


  # Calculate pred time
  end_time = timer()
  pred_time = round(end_time - start_time, 4)

  # Return pred dict and pred time
  return pred_labels_and_probs, pred_time

In [ ]:
print(predict("It's bad"))

tensor([[ 2.5530, -2.1476]])
({'bad': 0.9909921884536743, 'Good': 0.009007807821035385}, 1.0596)


In [ ]:
try:
  import gradio as gr
except:
  !pip -q install gradio
  import gradio as gr

print(f"Gradio version : {gr.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr

# Create title, description and article
title = "Good or Bad"
description = "Using XLMR_BASE_ENCODER"
# article = "Created at [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/#74-building-a-gradio-interface)"

# Create the gradio demo
demo = gr.Interface(
    fn = predict, # maps inputs to outputs
    inputs = "textbox",
    outputs=[
        gr.Label(num_top_classes=2, label="Predictions"),
        gr.Number(label = "Prediction time(s) ")
    ],
    title = title,
    description = description,
    # article = article
)

# Launch the demo !
demo.launch(
    debug = False, # print errors locally ?
    share = True
)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://adf83851-c693-43aa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [ ]:
import shutil
from pathlib import Path

# Create path
Sentiment_analysis_path = Path("demos/Sentiment_analysis")

# Remove files that might exist and create a new directory
if Sentiment_analysis_path.exists():
  shutil.rmtree(Sentiment_analysis_path)
  Sentiment_analysis_path.mkdir(
      parents = True,
      exist_ok = True
  )
else:
  Sentiment_analysis_path.mkdir(
      parents = True,
      exist_ok = True
  )

!ls demos/Sentiment_analysis/

In [ ]:
import shutil

# Create a source path for our target model
xlmr_base_encoder_model_path = "models/xlmr_base_encoder.pth"

# Create a destination path for our target model
xlmr_base_encoder_model_destination = Sentiment_analysis_path / xlmr_base_encoder_model_path.split("/")[1]

# Try to move the model file
try:
  print(f"[INFO] Attempting to move {xlmr_base_encoder_model_path} to {xlmr_base_encoder_model_destination}")

  # Move the model
  shutil.move(
      src = xlmr_base_encoder_model_path,
      dst = xlmr_base_encoder_model_destination
  )
  print(f"[INFO] Model move complete.")
# If the model has already been moved, check if it exists
except:
  print(f"[INFO] No model found at {xlmr_base_encoder_model_path}, perhaps its already been moved?")
  print(f"[INFO] Model exist at {xlmr_base_encoder_model_destination} : {xlmr_base_encoder_model_destination.exists()}")

[INFO] Attempting to move models/xlmr_base_encoder.pth to demos/Sentiment_analysis/xlmr_base_encoder.pth
[INFO] Model move complete.


In [ ]:
%%writefile demos/Sentiment_analysis/requirements.txt
torch == 1.13.1
torchvision == 0.14.1
torchdata == 0.5.1
gradio == 3.1.4

Writing demos/Sentiment_analysis/requirements.txt


In [ ]:
%%writefile demos/Sentiment_analysis/model.py
import torch
import torchtext
from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER
from torch import nn
def xlmr_base_encoder_model(num_classes:int=2, # default output classes = 2 (Bad, Good)):
  # 1, 2, 3 Create EffNetB2 pretrained weights, transforms and model
  transforms = torchtext.models.XLMR_BASE_ENCODER.transform()
  classifier_head = torchtext.RobertaClassificationHead(num_classes = 2, input_dim = 768)
  model = XLMR_BASE_ENCODER.get_model(head = classifier_head)

  # 4. Freeze all layers in the base model
  for param in model.parameters():
    param.requires_grad = False


  return model, transforms

Writing demos/Sentiment_analysis/model.py


In [ ]:
%%writefile demos/Sentiment_analysis/app.py

### 1. Imports and class names setup ###
import gradio as gr
import os
import torch
import torchtext
from model import xlmr_base_encoder_model
from timeit import default_timer as timer
from torchdata.datapipes.iter import IterableWrapper
from torch.utils.data import DataLoader
import torchtext.functional as F

# Setup class names
class_names = ["Bad", "Good"]

### 2. Model and transforms preparation ###
model, transforms = xlmr_base_encoder_model(
  num_classes = 2
)

# load save weights
model.load_state_dict(
  torch.load(
    f = "xlmr_base_encoder.pth",
    map_location = torch.device("cpu") # Load the model to the CPU
  )
)

### 3. Predict function ###

def predict(string):

  start_time = timer()

  var = (string, -9999999)
  dp = IterableWrapper([var])
  dp = dp.sharding_filter()

  padding_idx = 1
  bos_idx = 0
  eos_idx = 2
  max_seq_len = 256
  xlmr_vocab_path = r"https://download.pytorch.org/models/text/xlmr.vocab.pt"
  xlmr_spm_model_path = r"https://download.pytorch.org/models/text/xlmr.sentencepiece.bpe.model"

  text_transform = T.Sequential(
      T.SentencePieceTokenizer(xlmr_spm_model_path),
      T.VocabTransform(load_state_dict_from_url(xlmr_vocab_path)),
      T.Truncate(max_seq_len-2),
      T.AddToken(token = bos_idx, begin = True),
      T.AddToken(token = eos_idx, begin = False)
  )

  # Transform the raw dataset using non-batched API (i.e apply transformation line by line)
  def apply_transform(x):
    return transform(x[0]), x[1]

  dp = dp.map(apply_transform)
  dp = dp.batch(1)
  dp = dp.rows2columnar(["token_ids", "target"])
  dp = DataLoader(dp, batch_size=None)

  val = next(iter(dp))
  model.to('cpu')
  value = F.to_tensor(val["token_ids"], padding_value = padding_idx).to('cpu')
    # Pass transformed image through the model and turn the prediction logits into probabilities
  model.eval()
  with torch.inference_mode():
    answer = model(value)
  print(answer)
  # answer = answer.argmax(1)
  answer = torch.softmax(answer, dim=1)
  pred_labels_and_probs = {class_names[i]: float(answer[0][i]) for i in range(len(class_names))}


  # Calculate pred time
  end_time = timer()
  pred_time = round(end_time - start_time, 4)

  # Return pred dict and pred time
  return pred_labels_and_probs, pred_time

### 4. Gradio app ###
title = "Good or Bad"
description = "Using XLMR_BASE_ENCODER"

# Create the gradio demo
demo = gr.Interface(
    fn = predict, # maps inputs to outputs
    inputs = "textbox",
    outputs=[
        gr.Label(num_top_classes=2, label="Predictions"),
        gr.Number(label = "Prediction time(s) ")
    ],
    title = title,
    description = description,
    # article = article
)

# launch the demo!
demo.launch()

Writing demos/Sentiment_analysis/app.py


In [ ]:
!cd demos/Sentiment_analysis && zip -r ../Sentiment_analysis.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

  adding: app.py (deflated 54%)
  adding: model.py (deflated 48%)
  adding: requirements.txt (deflated 23%)
  adding: xlmr_base_encoder.pth (deflated 29%)


In [ ]:
# Download
try:
  from google.colab import files
  files.download("demos/Sentiment_analysis.zip")
except:
  print(f"Not running in Google Colab, can't use google.colab.files.download(), please download Sentiment_analysis.zip manually.")

Not running in Google Colab, can't use google.colab.files.download(), please download Sentiment_analysis.zip manually.
